In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [19]:
from fastai.vision import *
from semantic_segmentation_tile import *
from classic_segmentation import ImageDivider
from ipywidgets import interact
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20,16)

Load pretrained models

In [28]:
segment = SegmentationTilePredictor("models", "segmentation_resnet18_10_sept_2020.pkl", (233, 205), scale=2)

In [8]:
#load trained model
classify = load_learner("models", "Classification_resnet18_10_sept_2020.pkl")

Function for making predictions

In [57]:
colors = ['purple', 'yellow']
cm = matplotlib.colors.LinearSegmentedColormap.from_list(
        "mask", colors, N=2)

In [58]:
def predict(img_path):
    pred_mask, img = segment.predict_mask(img_path)
    imgs = ImageDivider.from_fastai(img, pred_mask, clean_mask=True).images()
    preds = pd.DataFrame([classify.predict(Image(pil2tensor(img / 255, np.float32))) for img in imgs], columns=['fruit', 'label', 'p'])
    p = preds.p.apply(max).mean()
    fruit = preds.fruit.mode().item()
    print(f"the selected image contains {len(imgs)} of {fruit} correctly classified with a probability of {round(p*100, 1)}%")
    fig, axes = plt.subplots(1,2)
    img.show(ax=axes[0], title="original image")
    pred_mask.show(ax=axes[1], title='predicted mask', cmap=cm)
    

In [59]:
interact(predict, img_path=get_image_files("dataset_segmentation/images"));

interactive(children=(Dropdown(description='img_path', options=(PosixPath('dataset_segmentation/images/Banana4…